## I. Table with the different dimensions and alphas for different models model_results.csv


In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns


### Adding Population ID (NH, CI) for the output csv files

In [3]:
'''Placing Population ID (NH, CI) for the output csv files'''

''' group_dict for a dictionary with participant id as key and grouping as values'''


participant_demographic = pd.read_csv("../data/raw_data/latest_demo.csv")

ID = participant_demographic['ID'].tolist()
Group = participant_demographic["Group"].tolist()
group_dict = dict(zip(ID, Group))

print(group_dict)

{211: 'CI', 234: 'CI', 300: 'CI', 308: 'CI', 310: 'CI', 312: 'CI', 320: 'CI', 329: 'CI', 335: 'CI', 338: 'CI', 344: 'CI', 390: 'CI', 450: 'CI', 452: 'CI', 483: 'CI', 520: 'CI', 544: 'CI', 556: 'CI', 558: 'CI', 563: 'CI', 570: 'CI', 587: 'CI', 672: 'NH', 673: 'NH', 674: 'NH', 681: 'NH', 698: 'NH', 702: 'NH', 707: 'NH', 711: 'NH', 715: 'NH', 720: 'NH', 722: 'NH', 724: 'NH', 726: 'NH', 730: 'NH', 744: 'NH', 753: 'NH', 754: 'NH', 759: 'NH', 764: 'NH', 767: 'NH', 781: 'CI', 721: 'NH', 657: 'NH', 666: 'NH', 671: 'NH', 686: 'NH', 691: 'NH', 693: 'NH', 697: 'NH', 663: 'NH', 678: 'NH', 498: 'CI', 539: 'CI', 340: 'CI', 436: 'CI', 401: 'CI', 315: 'CI', 159: 'CI'}


In [4]:
'''adding group column into the csv file'''
dimensions = ['50', '100', '200', '300']
type = [
    'alpha_0_s2v', 
    'alpha_0_w2v', 
    'alpha_0.1_s2v', 
    'alpha_0.1_w2v', 
    'alpha_0.2_s2v', 
    'alpha_0.2_w2v', 
    'alpha_0.3_s2v', 
    'alpha_0.3_w2v', 
    'alpha_0.4_s2v', 
    'alpha_0.4_w2v', 
    'alpha_0.5_s2v', 
    'alpha_0.5_w2v', 
    'alpha_0.6_s2v', 
    'alpha_0.6_w2v',
    'alpha_0.7_s2v', 
    'alpha_0.7_w2v', 
    'alpha_0.8_s2v', 
    'alpha_0.8_w2v', 
    'alpha_0.9_s2v', 
    'alpha_0.9_w2v', 
    'alpha_1.0_s2v', 
    'alpha_1.0_w2v',
    'average',
    'only_s2v', 
    'only_w2v'
]


for dim in dimensions: 
    for t in type:         
        
        path = '../forager/output/' + dim + '_dim_results/' + t + '_results' +'/model_results.csv'
        forager_ind_stats = pd.read_csv(path)
        #get the list of subjects and create a new list for their groupings 
        #word2vec first 
        forager_subjects = forager_ind_stats["Subject"].tolist()
        forager_groupings = [] 

        for index, row in forager_ind_stats.iterrows(): 
            forager_groupings += [group_dict[int(row['Subject'][4:])]]

        forager_ind_stats['Group'] = forager_groupings

        forager_ind_stats.to_csv(path, index = False)


### Model_results dataframe with alpha, average, and dimensions 

In [21]:
#model_results dataframe => dimension_model_results 
dimension_model_results = pd.DataFrame(columns=["dimension", "type", "model", "group", "sum_NLL", "mean_beta_semantic", "mean_beta_freq", "mean_beta_phon"])

dimensions = ['50', '100', '200', '300']
type = [
    'alpha_0_s2v', 
    'alpha_0_w2v', 
    'alpha_0.1_s2v', 
    'alpha_0.1_w2v', 
    'alpha_0.2_s2v', 
    'alpha_0.2_w2v', 
    'alpha_0.3_s2v', 
    'alpha_0.3_w2v', 
    'alpha_0.4_s2v', 
    'alpha_0.4_w2v', 
    'alpha_0.5_s2v', 
    'alpha_0.5_w2v', 
    'alpha_0.6_s2v', 
    'alpha_0.6_w2v',
    'alpha_0.7_s2v', 
    'alpha_0.7_w2v', 
    'alpha_0.8_s2v', 
    'alpha_0.8_w2v', 
    'alpha_0.9_s2v', 
    'alpha_0.9_w2v', 
    'alpha_1.0_s2v', 
    'alpha_1.0_w2v',
    'average',
    'only_s2v', 
    'only_w2v'
]


models = [
    'forage_static',
    'forage_dynamic_simdrop',
    'forage_phonologicalstatic',
    'forage_phonologicaldynamicglobal_simdrop',
    'forage_phonologicaldynamiclocal_simdrop',
    'forage_phonologicaldynamicswitch_simdrop',
    'forage_random_baseline'
]

groups = ["NH", "CI"]
stats = ["sum_NLL", "mean_beta_semantic", "mean_beta_freq", "mean_beta_phon"]
stats_label = ["Negative_Log_Likelihood_Optimized", "Beta_Semantic", "Beta_Frequency", "Beta_Phonological"]

rows= [] 

for dim in dimensions: 
    for t in type: 
        path = '../forager/output/' + dim + '_dim_results/' + t + '_results' +'/model_results.csv'

        data = pd.read_csv(path)
        data_groups = data.groupby(["Group", "Model"])

        for group in groups: 
            for model in models: 
                row = [dim, t, model, group]
                test_data = data_groups.get_group((group, model))

                j = 0 
                while j < len(stats): 
                    if j == 0:
                        row += [test_data[stats_label[j]].sum()]
                    else: 
                        row += [test_data[stats_label[j]].mean()]
                    j += 1 
                rows += [row]


print(rows[0])
for line in rows: 
    dimension_model_results.loc[len(dimension_model_results)] = line 
dimension_model_results.to_csv("../results/model_results.csv", index = False)
dimension_model_results



['50', 'alpha_0_s2v', 'forage_static', 'NH', 3579.2110627955767, 2.3096849301862488, 8.063681495930794, nan]


,dimension,type,model,group,sum_NLL,mean_beta_semantic,mean_beta_freq,mean_beta_phon
0,50,alpha_0_s2v,forage_static,NH,3579.211063,2.309685,8.063681,NaN
1,50,alpha_0_s2v,forage_dynamic_simdrop,NH,3488.038621,3.919814,8.013671,NaN
2,50,alpha_0_s2v,forage_phonologicalstatic,NH,3567.164725,2.329063,8.097019,-0.002322
3,50,alpha_0_s2v,forage_phonologicaldynamicglobal_simdrop,NH,3475.569955,3.996996,8.058893,-0.012554
4,50,alpha_0_s2v,forage_phonologicaldynamiclocal_simdrop,NH,3474.639359,3.957541,8.026180,-0.003894
...,...,...,...,...,...,...,...,...
1395,300,only_w2v,forage_phonologicalstatic,CI,3002.663683,1.911552,9.138641,-0.030755
1396,300,only_w2v,forage_phonologicaldynamicglobal_simdrop,CI,2945.467222,2.702767,8.998706,-0.045976
1397,300,only_w2v,forage_phonologicaldynamiclocal_simdrop,CI,2939.119670,2.857655,9.049068,-0.082495
1398,300,only_w2v,forage_phonologicaldynamicswitch_simdrop,CI,2952.983589,2.488560,8.888333,-0.229952


## II. Best NLL for dimensions and models 

In [24]:
"""50 dimension NLL best alpha value for each model and group"""

models = [
    'forage_static',
    'forage_dynamic_simdrop',
    'forage_phonologicalstatic',
    'forage_phonologicaldynamicglobal_simdrop',
    'forage_phonologicaldynamiclocal_simdrop',
    'forage_phonologicaldynamicswitch_simdrop',
    'forage_random_baseline'
]

groups = ["NH", "CI"]

dimensional_model_results = pd.read_csv("../results/model_results.csv")

dimensional_model_results

df = dimensional_model_results.groupby(["dimension", "model", "group"])

dim_nll = pd.DataFrame(columns=["dimension","type", "model", "group", "sum_NLL", "mean_beta_semantic", "mean_beta_freq", "mean_beta_phon"])

for group in groups: 
    for model in models: 
        test_data = df.get_group((50, model, group))
        dim_nll = pd.concat([dim_nll, test_data.nsmallest(1, 'sum_NLL')])

dim_nll.reset_index(drop=True, inplace=True)
dim_nll.to_csv("../results/dim_NLL.csv", index = False)

dim_nll


/var/folders/s7/md5b66vn4l39df_j063nqlp00000gn/T/ipykernel_23141/2177503728.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dim_nll = pd.concat([dim_nll, test_data.nsmallest(1, 'sum_NLL')])


,dimension,type,model,group,sum_NLL,mean_beta_semantic,mean_beta_freq,mean_beta_phon
0,50,alpha_0.5_s2v,forage_static,NH,3563.394123,2.571709,8.100351,NaN
1,50,alpha_0.5_s2v,forage_dynamic_simdrop,NH,3480.017431,4.059364,7.995946,NaN
2,50,alpha_0.5_s2v,forage_phonologicalstatic,NH,3551.374325,2.594799,8.133001,-0.004351
3,50,alpha_0.5_w2v,forage_phonologicaldynamicglobal_simdrop,NH,3467.175546,4.148725,8.033588,-0.014505
4,50,alpha_0.5_w2v,forage_phonologicaldynamiclocal_simdrop,NH,3465.206654,4.119059,8.019748,-0.006805
5,50,alpha_0.5_w2v,forage_phonologicaldynamicswitch_simdrop,NH,3462.110556,4.060022,8.018517,-0.167144
6,50,alpha_0_s2v,forage_random_baseline,NH,3989.522585,0.000000,0.000000,NaN
7,50,alpha_0.4_w2v,forage_static,CI,2974.806833,3.322440,9.003586,NaN
8,50,alpha_0.4_w2v,forage_dynamic_simdrop,CI,2914.652050,4.713965,8.706446,NaN
9,50,alpha_0.4_w2v,forage_phonologicalstatic,CI,2953.880336,3.428166,9.026273,-0.014260


In [23]:
"""NLL best alpha value for each model and group"""

models = [
    'forage_static',
    'forage_dynamic_simdrop',
    'forage_phonologicalstatic',
    'forage_phonologicaldynamicglobal_simdrop',
    'forage_phonologicaldynamiclocal_simdrop',
    'forage_phonologicaldynamicswitch_simdrop',
    'forage_random_baseline'
]

groups = ["NH", "CI"]

dimensional_model_results = pd.read_csv("../results/model_results.csv")

dimensional_model_results

df = dimensional_model_results.groupby(["model", "group"])

min_nll = pd.DataFrame(columns=["dimension","type", "model", "group", "sum_NLL", "mean_beta_semantic", "mean_beta_freq", "mean_beta_phon"])


for group in groups: 
    for model in models: 
        test_data = df.get_group((model, group))
        min_nll = pd.concat([min_nll, (test_data.nsmallest(1, 'sum_NLL'))])

min_nll.reset_index(drop=True, inplace=True)
min_nll


/var/folders/s7/md5b66vn4l39df_j063nqlp00000gn/T/ipykernel_23141/4057422110.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  min_nll = pd.concat([min_nll, (test_data.nsmallest(1, 'sum_NLL'))])


,dimension,type,model,group,sum_NLL,mean_beta_semantic,mean_beta_freq,mean_beta_phon
0,50,alpha_0.5_s2v,forage_static,NH,3563.394123,2.571709,8.100351,NaN
1,50,alpha_0.5_s2v,forage_dynamic_simdrop,NH,3480.017431,4.059364,7.995946,NaN
2,50,alpha_0.5_s2v,forage_phonologicalstatic,NH,3551.374325,2.594799,8.133001,-0.004351
3,50,alpha_0.5_w2v,forage_phonologicaldynamicglobal_simdrop,NH,3467.175546,4.148725,8.033588,-0.014505
4,50,alpha_0.5_w2v,forage_phonologicaldynamiclocal_simdrop,NH,3465.206654,4.119059,8.019748,-0.006805
5,50,alpha_0.5_w2v,forage_phonologicaldynamicswitch_simdrop,NH,3462.110556,4.060022,8.018517,-0.167144
6,50,alpha_0_s2v,forage_random_baseline,NH,3989.522585,0.000000,0.000000,NaN
7,50,alpha_0.4_w2v,forage_static,CI,2974.806833,3.322440,9.003586,NaN
8,50,alpha_0.4_w2v,forage_dynamic_simdrop,CI,2914.652050,4.713965,8.706446,NaN
9,50,alpha_0.4_w2v,forage_phonologicalstatic,CI,2953.880336,3.428166,9.026273,-0.014260
